In [1]:
# Cargamos las librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df= pd.read_csv('itembyticket.csv')
del df['COUNT']
df.head()

,TransId,ItemCode
0,410022557219348320160423,3349668514830
1,140027954620190510123738,3331840107169
2,48002852897181120160813,3274872300583
3,13008651920181221205242,3349668543021
4,13008716920190109111031,0ATG0B


In [2]:
group = df.groupby(["TransId","ItemCode"])["ItemCode"].count()
group.head(30)

TransId                   ItemCode     
100110732220170807130846  8411061818961    1
10025291520181031171328   8426017055710    1
                          8426017057769    1
100510628620170927104604  3147758235500    1
100515225320200814141635  3145891235807    1
                          3346470304345    1
                          719346627276     1
100515247520200921141632  3274872346000    1
100515247820200922131533  8411061926093    1
100515251020200928143352  3145891265200    1
                          873824001108     1
100515251420200929122740  3145891073706    1
100515253220201001135817  022548040126     1
                          763511009824     1
10059432620170306135152   088300107438     1
                          8011003814367    1
10081649520200130175204   3348901211512    1
                          3348901407243    1
                          3423473020615    1
                          3423478525917    1
10083766820200130175206   3346470109537    1
               

## Transformación

Transformando las listas dentro de otras listas de las columnas

In [3]:
g = df.groupby('TransId').cumcount()
mux = pd.MultiIndex.from_product([df['TransId'].unique(), g.unique()])
transactions = (df.set_index(['TransId',g])
       .reindex(mux, fill_value='nan')
       .groupby(level=0)['ItemCode']
       .apply(lambda x: x.values.tolist())
       .tolist()
)
transactions

##print(transactions[1])

[['8411061818961',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['8426017057769',
  '8426017055710',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan

In [4]:
final_data = []
i=0
for transaction in transactions:
    cleanedList = [x for x in transactions[i] if str(x) != 'nan']
    final_data.append(cleanedList)
    i += 1
    
    
##final_data

## Obtención de reglas de asociación

Ahora se pude utilizar la función `apriori` para obtener los productos con un mínimo de soporte y todas las características que necesitamos en los hiperparámetros.


In [47]:
from apyori import apriori

rules = apriori(final_data, min_support = 0.001, min_confidence = 0.2, min_lift = 3, min_length = 2)  

In [48]:
results = list(rules)

print(results)


[RelationRecord(items=frozenset({'8410225071006', '8410225518235'}), support=0.0010196412334505394, ordered_statistics=[OrderedStatistic(items_base=frozenset({'8410225071006'}), items_add=frozenset({'8410225518235'}), confidence=0.3310580204778157, lift=67.36694347611834), OrderedStatistic(items_base=frozenset({'8410225518235'}), items_add=frozenset({'8410225071006'}), confidence=0.20748663101604278, lift=67.36694347611834)]), RelationRecord(items=frozenset({'8410225524588', '8410225518235'}), support=0.0013547037521746215, ordered_statistics=[OrderedStatistic(items_base=frozenset({'8410225518235'}), items_add=frozenset({'8410225524588'}), confidence=0.275668449197861, lift=50.61472246053812), OrderedStatistic(items_base=frozenset({'8410225524588'}), items_add=frozenset({'8410225518235'}), confidence=0.24873341375150787, lift=50.614722460538125)]), RelationRecord(items=frozenset({'8410225529613', '8410225518235'}), support=0.0012417022752715977, ordered_statistics=[OrderedStatistic(ite

In [49]:
#results[0:2]

In [50]:
def inspect(results):
    rh          = [tuple(result[2][0][0]) for result in results]
    lh          = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(rh, lh, supports, confidences, lifts))

In [51]:
# Este comamdo crea un frame para ver los datos resultados
resultDataFrame=pd.DataFrame(inspect(results),
                columns=['X','Y','support','confidence','lift'])

In [52]:
resultDataFrame.sort_values(by='lift',ascending= False)

,X,Y,support,confidence,lift
7,"(8411061705919,)","(8411061715482,)",0.001423,0.272384,74.061672
0,"(8410225071006,)","(8410225518235,)",0.001020,0.331058,67.366943
3,"(8410225524588,)","(8410225529613,)",0.001547,0.283957,65.845740
2,"(8410225518235,)","(8410225529613,)",0.001242,0.252674,58.591681
1,"(8410225518235,)","(8410225524588,)",0.001355,0.275668,50.614722
4,"(8411061771907,)","(8411061027011,)",0.003669,0.584712,30.651348
5,"(8411061849330,)","(8411061027011,)",0.003247,0.536125,28.104352
6,"(8411061401705,)","(8411061747261,)",0.006762,0.320025,15.194683
